In [3]:
from agno.agent import Agent
from agno.team import Team
from agno.tools import tool
from agno.models.openai import OpenAIChat
from agno.knowledge.pdf import PDFKnowledgeBase
from agno.vectordb.qdrant import Qdrant
from agno.vectordb.search import SearchType
from dotenv import load_dotenv
import os
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
# === Tool-Based Agents ===

@tool(name="clickup_tool", stop_after_tool_call=False )
def clickup_tool(**kwargs):
    return "✅ ClickUp tool triggered"

clickup_agent = Agent(
    name="ClickUp Agent",
    model=OpenAIChat(id="gpt-4.1-nano"),
    tools=[clickup_tool],
    show_tool_calls=True,
    read_chat_history=True,
    instructions=["Only respond to ClickUp-related tasks like creating, updating, or assigning tasks."]
)

@tool(name="slack_tool", stop_after_tool_call=False)
def slack_tool(**kwargs):
    return "✅ Slack tool triggered"

slack_agent = Agent(
    name="Slack Agent",
    model=OpenAIChat(id="gpt-4.1-nano"),
    tools=[slack_tool],
    show_tool_calls=True,
    read_chat_history=True,
    instructions=["Handle Slack messaging, tagging, and notifications."]
)

@tool(name="github_tool", stop_after_tool_call=False )
def github_tool(**kwargs):
    return "✅ GitHub tool triggered"

github_agent = Agent(
    name="GitHub Agent",
    model=OpenAIChat(id="gpt-4.1-nano"),
    tools=[github_tool],
    show_tool_calls=True,
    read_chat_history=True,
    instructions=["Respond to GitHub requests like creating issues, reviewing PRs, or commenting."]
)

@tool(name="devin_tool", stop_after_tool_call=False )
def devin_tool(**kwargs):
    return "✅ Devin tool triggered"

devin_agent = Agent(
    name="Devin Agent",
    model=OpenAIChat(id="gpt-4.1-nano"),
    tools=[devin_tool],
    show_tool_calls=True,
    read_chat_history=True,
    instructions=["Handle developer tasks like debugging, running tests, or building workflows."]
)

# === RAG Agent ===

collection_name = "new"
qdrant_url = "http://localhost:6333"

knowledge_base = PDFKnowledgeBase(
                path=f"data/user_abc",
                vector_db=Qdrant(
                    collection=collection_name,
                    url=qdrant_url,
                    # search_type=SearchType.hybrid,
                ),
            )

rag_agent = Agent(
    name="Knowledge Agent",
    model=OpenAIChat(id="gpt-4.1-nano"),
    knowledge=knowledge_base,
    search_knowledge=True,
    read_chat_history=True,
    instructions=[
            "You are a helpful assistant that can answer questions based on the provided PDF documents.",
            "Always cite the source when providing information from the documents.",
            "If you cannot find the answer in the documents, clearly state that.",
            "Be concise but thorough in your responses.",
            "Cite sources when possible. If nothing is found, say so clearly."
        ],
)
# === Orchestrator Team ===

orchestrator = Team(
    name="XO Orchestrator",
    model=OpenAIChat(id="gpt-4o"),
    members=[
        rag_agent,
        clickup_agent,
        slack_agent,
        github_agent,
        devin_agent
    ],
    instructions = [
                        "You are the coordinator for a team of specialized agents, each responsible for handling a specific type of request.",
                        
                        "You must determine which agent to delegate a user message to, based on the presence of explicit `@mentions`.",
                        
                        "Only trigger a specialized tool agent if the message includes one of the exact @mentions listed below. "
                        "If no such @mention is present, route the message to the `rag_agent` by default.",
                        
                        "Agent routing rules:",
                        "- Route to `clickup_agent` only if the message contains `@click-up`.",
                        "- Route to `slack_agent` only if the message contains `@slack`.",
                        "- Route to `github_agent` only if the message contains `@github`.",
                        "- Route to `copilot_agent` only if the message contains `@github-copilot`.",
                        "- Route to `devin_agent` only if the message contains `@devin`.",
                        "- Always route to `rag_agent` if the message contains `@XO`, or if no @mention is present at all.",
                        
                        "Do not infer tool usage from intent or keywords alone. If a message talks about GitHub or Slack but does not include the corresponding @mention, it should go to the `rag_agent`.",
                        
                        "If the message is ambiguous, vague, or lacking required context, delegate to the `rag_agent` and allow it to respond or request clarification.",
                        
                        "Your role is strictly to select the correct agent for execution. You do not generate or modify responses yourself."
                    ],
    mode="route",
    share_member_interactions=True,
    show_members_responses=True,
    read_team_history=True,
    enable_agentic_context=True,
    enable_team_history=True,
    num_of_interactions_from_history=5,
    show_tool_calls=True,
)
knowledge_base.load(recreate=False)

# === Example Usage ===

# if __name__ == "__main__":
#     response = orchestrator.run("Update the ClickUp task to fix the login bug and notify @krish on Slack.")
#     print(response)


INFO Embedder not provided, using OpenAIEmbedder as default.

INFO Creating collection

INFO Loading knowledge base

INFO Reading: Akreage Launch Pad Whitepaper

INFO Added 6 documents to knowledge base

INFO Reading: Akreage_Launchpad_Scope_of_Work (1)

INFO Added 2 documents to knowledge base

INFO Reading: Plan of Action

INFO Added 5 documents to knowledge base

In [4]:
def chat_with_orchestrator(agent):
    print("🧠 XO Orchestrator is ready. Type 'exit' to quit.\n")
    while True:
        user_input = input("You: ").strip()
        if user_input.lower() in ["exit", "quit"]:
            print("👋 Goodbye!")
            break
        try:
            agent.print_response(user_input,stream=True)
            # response = agent.run(user_input)
            # print(f"🧠 XO: {response.content}\n")
        except Exception as e:
            print(f"❌ Error: {e}\n")
if __name__ == "__main__":
    chat_with_orchestrator(orchestrator)

🧠 XO Orchestrator is ready. Type 'exit' to quit.



Output()

Output()

Output()

👋 Goodbye!
